In [1]:
import pandas as pd
import soynlp
from soynlp.noun import LRNounExtractor_v2
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer

import warnings
warnings.filterwarnings('ignore')

In [2]:
def clean_sents_df(company):
    try:
        target = pd.read_csv(f'./src/2year_time/{company}_2year.csv',encoding='utf8').drop('Unnamed: 0',axis=1)
    except:
        target = pd.read_csv(f'./src/2year_time/{company}_2year.csv',encoding='utf8')

    if company == 'sampro':
        target.rename(columns={'comment':'댓글'},inplace=True)
    df = target
    df['정제된 댓글'] = df['댓글'].str.replace('\\[삭제된 게시물의 답글\\]',' ')
    df['정제된 댓글'] = df['정제된 댓글'].str.replace('답글:',' ')
    df['정제된 댓글'] = df['정제된 댓글'].str.replace('[^가-힣]',' ').str.replace(' +',' ').str.strip()
    df = df[df['정제된 댓글'] != '']
    df = df.reset_index(drop=True)
    return  df

def corpus_save(company):
    df = clean_sents_df(company)
    df['정제된 댓글 길이'] = [len(str(i)) for i in df['정제된 댓글']]
    df = df[df['정제된 댓글 길이'] > 5]

    tp = [str(i) for i in list(df['정제된 댓글'])]
    save = '\n'.join(tp)
    f = open("./corpus_target.txt", 'a',encoding='utf8')
    f.write(save)
    f.close()

    
def corpus_init():
    ktop30_company = pd.read_excel('./KODEX_KTOP_30_20220629.xlsx',header=2).drop(0,axis=0)['종목명']
    company_set = list(ktop30_company)
    company_set.append('sampro')
    f = open("./corpus_target.txt", 'w',encoding='utf8')
    f.write('')
    f.close()
    for company in company_set:
        corpus_save(company)

def return_tokenizer():
    corpus = DoublespaceLineCorpus("./corpus_target.txt",iter_sent=True)
    noun_extractor = LRNounExtractor_v2(verbose=True)
    nouns = noun_extractor.train_extract(corpus)
    scores = {word:score.score for word, score in nouns.items()}
    tokenizer = LTokenizer(scores=scores)
    return tokenizer

In [3]:
corpus_init()
tokenizer = return_tokenizer()

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 1249734 from 2753662 sents. mem=0.333 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=9746776, mem=2.907 Gb
[Noun Extractor] batch prediction was completed for 289636 words
[Noun Extractor] checked compounds. discovered 280851 compounds
[Noun Extractor] postprocessing detaching_features : 249805 -> 199708
[Noun Extractor] postprocessing ignore_features : 199708 -> 199316
[Noun Extractor] postprocessing ignore_NJ : 199316 -> 196316
[Noun Extractor] 196316 nouns (280851 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=3.316 Gb                    
[Noun Extractor] 71.68 % eojeols are covered


In [4]:
target_df = clean_sents_df('삼성전자')
target_df['토큰화 댓글'] = [tokenizer(str(i)) for i in target_df['정제된 댓글']]
target_df

,날짜,댓글,조회수,좋아요,싫어요,정제된 댓글,토큰화 댓글
0,2022-07-11 23:42,요즘 한전 게시판 조용하네!ㅋ,2,0,0,요즘 한전 게시판 조용하네,"[요즘, 한전, 게시판, 조용, 하네]"
1,2022-07-11 23:42,1% 올려야 함,4,0,0,올려야 함,"[올려야, 함]"
2,2022-07-11 23:42,=자신들잘못된죄들=모르는=내로남불들=척결...,3,0,0,자신들잘못된죄들 모르는 내로남불들 척결,"[자신들, 잘못된죄들, 모르는, 내로남불, 들, 척결]"
3,2022-07-11 23:42,난 내일 가요,9,0,0,난 내일 가요,"[난, 내일, 가요]"
4,2022-07-11 23:42,실적으로 이만한 실적내는 기업 있으면 나...,6,0,0,실적으로 이만한 실적내는 기업 있으면 나,"[실적, 으로, 이만, 한, 실적, 내는, 기업, 있으면, 나]"
...,...,...,...,...,...,...,...
952447,2020-06-01 00:09,내8자,707,1,3,내 자,"[내, 자]"
952448,2020-06-01 00:08,하락낙폭저지 의 길,682,2,0,하락낙폭저지 의 길,"[하락낙폭저지, 의, 길]"
952449,2020-06-01 00:07,주주님들 오랜만에 오르니까 좋네요 ㅎㅎㅎ,697,1,0,주주님들 오랜만에 오르니까 좋네요,"[주주님, 들, 오랜만, 에, 오르니까, 좋네요]"
952450,2020-06-01 00:03,코덱스200선물인버스 곱배기..,725,0,0,코덱스 선물인버스 곱배기,"[코덱스, 선물, 인버스, 곱배기]"


# 레이블링 & 랜덤추출

---

In [5]:
def labeling(target_df):
    
    f = open("./src/fear_words.txt", 'r',encoding='utf8')
    words = f.readlines()
    f.close()
    fear_words_set = {word.strip('\n') for word in words}

    f = open("./src/greed_words.txt", 'r',encoding='utf8')
    words = f.readlines()
    f.close()
    greed_words_set = {word.strip('\n') for word in words}
    
    label_score = []
    for token_list in target_df['토큰화 댓글']:
        sent_score = 0
        for token in token_list:
            if token in fear_words_set:
                sent_score -= 1
            elif token in greed_words_set:
                sent_score += 1

        if sent_score < 0:
            label_score.append(-1)
        elif sent_score > 0:
            label_score.append(1)
        else:
            label_score.append(0)
            
    target_df['label'] = label_score
    
    return target_df


In [6]:
def setting_train_data():
    company = list(pd.read_excel('./KODEX_KTOP_30_20220629.xlsx',header=2)['종목명'][1:])
    dominance = list(pd.read_excel('./KODEX_KTOP_30_20220629.xlsx',header=2)['비중(%)'][1:])

    train_data = pd.DataFrame(columns=['날짜','댓글','조회수','좋아요','싫어요','정제된 댓글','토큰화 댓글','label'])
    for idx in range(30):
        target_df = clean_sents_df(company[idx])
        target_df['토큰화 댓글'] = [tokenizer(str(i)) for i in target_df['정제된 댓글']]
        
        label_df = labeling(target_df)
        
        label_df = label_df[label_df['label'] != 0]
        label_df = label_df.sample(int(30000*dominance[idx])+1,replace=True) # replace option : 샘플수가 부족할경우 중복이 생김 ** 추후 수정
        train_data = train_data.append(label_df)
    
    return train_data

In [55]:
train_data = setting_train_data()
train_data.reset_index(inplace=True)
train_data.drop('index',axis=1,inplace=True)
train_data

,날짜,댓글,조회수,좋아요,싫어요,정제된 댓글,토큰화 댓글,label
742353,2021-01-19 10:00,지금이 가장저점 ! 내일고 오른다! 사라,213,2,1,지금이 가장저점 내일고 오른다 사라,"[지금, 이, 가장저점, 내일, 고, 오른다, 사라]",1
715345,2021-01-28 08:53,고...공포다! 더 사야합니까?,251,1,0,고 공포다 더 사야합니까,"[고, 공포, 다, 더, 사야, 합니까]",-1
920389,2020-07-08 10:37,"코로나-세계경기폭락-삼전,기업,개인-실적...",345,5,0,코로나 세계경기폭락 삼전 기업 개인 실적,"[코로나, 세계, 경기폭락, 삼전, 기업, 개인, 실적]",-1
109461,2022-06-12 21:19,공포의 월요일,257,2,1,공포의 월요일,"[공포, 의, 월요, 일]",-1
858930,2020-10-08 09:46,솔직히 실적발표나고나서,815,3,0,솔직히 실적발표나고나서,"[솔직, 히, 실적, 발표나고나서]",-1
...,...,...,...,...,...,...,...,...
10600,2020-11-22 22:29,국채매수 실적평가 프로그램을 빈번하게 가...,104,0,0,국채매수 실적평가 프로그램을 빈번하게 가,"[국채매수, 실적, 평가, 프로그램, 을, 빈번, 하게, 가]",-1
10113,2020-12-02 11:29,ㅋㅋㅋ 어제 정말 오랫만에 쫌 오른다 싶...,249,3,1,어제 정말 오랫만에 쫌 오른다 싶,"[어제, 정말, 오랫만, 에, 쫌, 오른다, 싶]",1
1906,2021-11-11 15:45,실적 나왔는데,609,12,1,실적 나왔는데,"[실적, 나왔는데]",-1
2040,2021-10-25 16:37,실적 좋고 챠트 좋고 스급 좋고,329,3,2,실적 좋고 챠트 좋고 스급 좋고,"[실적, 좋고, 챠트, 좋고, 스급, 좋고]",-1


---

In [57]:
# 삼프로
target_df = clean_sents_df('sampro')
target_df['토큰화 댓글'] = [tokenizer(str(i)) for i in target_df['정제된 댓글']]
label_df = labeling(target_df)

label_df = label_df[label_df['label'] != 0]
label_df = label_df.sample(10000,replace=True) # replace option : 샘플수가 부족할경우 중복이 생김 ** 추후 수정
train_data = train_data.append(label_df)

In [61]:
train_data = train_data.loc[:,['정제된 댓글','label']].reset_index().drop('index',axis=1)

In [62]:
train_data

,정제된 댓글,label
0,지금이 가장저점 내일고 오른다 사라,1
1,고 공포다 더 사야합니까,-1
2,코로나 세계경기폭락 삼전 기업 개인 실적,-1
3,공포의 월요일,-1
4,솔직히 실적발표나고나서,-1
...,...,...
40010,이노텍 실적 최고,-1
40011,삼프로 와 함께 성장해주셔서 정말 감사합니다 감사한 마음에 보답하고자 주간 삼프로 ...,-1
40012,만 구독 감사 이벤트는 본 영상에 댓글을 달아도 참여하실 수 있습니다 경품응모 신청...,1
40013,삼프로 가 준비한 다양한 프리미엄 콘텐츠 삼프로 년 한정판 다이어리 투자의 정답을 ...,1
